<a href="https://colab.research.google.com/github/nrally1/AIPS/blob/main/Spiral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing modules
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imutils import paths
import cv2 as cv2
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
%matplotlib inline
from IPython import display
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import zipfile
from skimage import feature
import os
import shutil
!pip install Django==3.2.6
from sklearn.metrics import confusion_matrix
import matplotlib.image
from matplotlib import pyplot

from numpy import expand_dims

! pip install -q kaggle

from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

! kaggle datasets download -d kmader/parkinsons-drawings

micrographia_dataset = '/content/parkinsons-drawings.zip'

! unzip parkinsons-drawings.zip

micrographia_files = zipfile.ZipFile(micrographia_dataset, 'r')
micrographia_files.extractall('/tmp')
micrographia_files.close()






/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.9 MB 22.6 MB/s 


Saving kaggle.json to kaggle.json
ref                                                 title                                  size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ------------------------------------  -----  -------------------  -------------  ---------  ---------------  
ruchi798/data-science-job-salaries                  Data Science Job Salaries               7KB  2022-06-15 08:59:12          18033        558  1.0              
nancyalaswad90/diamonds-prices                      Diamonds Prices                       711KB  2022-07-09 14:59:21           2008         91  1.0              
aravindas01/monkeypox-cases-countrywise-data        MonkeyPox Cases_Countrywise Data        6KB  2022-08-10 17:12:36            569         31  0.9117647        
faryarmemon/usa-housing-market-factors              U.S. Housing Market Factors            32KB  2022-08-03 02:19:31            449         31  1.0         

In [2]:
def quantify_image(image):
    features = feature.hog(image, orientations=9,
                           pixels_per_cell=(10, 10), cells_per_block=(2, 2),
                           transform_sqrt=True, block_norm="L1")
    return features

def load_split(path):
    # grab the list of images in the input directory, then initialize
    # the list of data (i.e., images) and class labels
    imagePaths = list(paths.list_images(path))
    data = []
    labels = []
    # loop over the image paths
    for imagePath in imagePaths:
        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]
        # load the input image, convert it to grayscale, and resize
        # it to 200x200 pixels, ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200, 200))
        # threshold the image such that the drawing appears as white
        # on a black background
        image = cv2.threshold(image, 0, 255,
                              cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        # quantify the image
        features = quantify_image(image)
        # update the data and labels lists, respectively
        data.append(features)
        labels.append(label)
    return (np.array(data), np.array(labels))


def train_models(dataset):
    # initialize the models
    model = {
            "classifier": RandomForestClassifier(random_state=1),
            "accuracy": 0,
            "sensitivity": 0,
            "specificity": 0,
    }
    # define the path to the testing and training directories
    path = "./" + dataset
    trainingPath = os.path.sep.join([path, "training"])
    testingPath = os.path.sep.join([path, "testing"])
    # load the data
    (trainX, trainY) = load_split(trainingPath)
    print(trainX.shape)
    print(trainY.shape)
    (testX, testY) = load_split(testingPath)
    print(testX.shape)
    print(testY.shape)
    # encode the labels
    le = LabelEncoder()
    trainY = le.fit_transform(trainY)
    testY = le.transform(testY)

    # train each model and calculate its accuracy
    model["classifier"].fit(trainX, trainY)
    predictions = model["classifier"].predict(testX)
    cm = confusion_matrix(testY, predictions).ravel()
    tn, fp, fn, tp = cm
    model["accuracy"] = (tp + tn) / float(cm.sum())
    model["sensitivity"] = tp / float(tp + fn)
    model["specificity"] = tn / float(tn + fp)
    return model

In [3]:
# Train the model on the spiral drawings
spiralModel = train_models('spiral')

for metric in ("accuracy", "sensitivity", "specificity"):
    print(f"{metric.capitalize()}: ")
    print("Random Forrest={:.2f}%".format(
        spiralModel[metric]*100))

(72, 12996)
(72,)
(30, 12996)
(30,)
Accuracy: 
Random Forrest=80.00%
Sensitivity: 
Random Forrest=73.33%
Specificity: 
Random Forrest=86.67%
